In [1]:
!nvidia-smi

Mon Dec  2 03:51:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
! pip install biopython pandas numpy matplotlib
! pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu121


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.9 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/

In [1]:
from Bio.PDB import PDBParser
import numpy as np
import pandas as pd
import os

import requests

def fetch_pdb(pdb_id):
    url = f'https://files.rcsb.org/download/{pdb_id}.pdb'
    os.system(f"wget {url}")

def convert_three_to_one_in_string(input_string):
    """
    Converts a string of concatenated 3-letter amino acid codes into a 1-letter sequence.

    Parameters:
        input_string (str): Concatenated string of 3-letter amino acid codes.

    Returns:
        str: Corresponding 1-letter amino acid sequence.
    """
    # Mapping of 3-letter codes to 1-letter codes
    mapping = {
        'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
        'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
        'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
        'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y',
        'SEC': 'U',  # Selenocysteine
        'PYL': 'O'   # Pyrrolysine
    }

    # Break the input string into chunks of 3 letters
    chunks = [input_string[i:i + 3] for i in range(0, len(input_string), 3)]

    # Convert each chunk to 1-letter using the mapping
    one_letter_sequence = ''.join(mapping.get(chunk.upper(), 'X') for chunk in chunks)

    return one_letter_sequence




In [7]:

def preprocessing(pdb_id):
  # pdb_id = '4hhb' # I chose this specific sequence/s, but it's possible to edit it and choose anything else
  fetch_pdb(pdb_id)

  os.system(f'mkdir data; mv {pdb_id}.pdb data')
  # Load protein structure
  pdb_file = f"./data/{pdb_id}.pdb"
  parser = PDBParser(QUIET=True)
  structure = parser.get_structure(pdb_id.upper(), pdb_file)

  sequences = {}
  for model in structure:
      for chain in model:
          # Get sequence for each chain
          sequence = "".join([res.get_resname() for res in chain if res.id[0] == " "])
          sequences[chain.id] = sequence

  # Calculate Cα distances
  ca_atoms = [atom for atom in structure.get_atoms() if atom.id == "CA"]
  n_atoms = len(ca_atoms)
  contacts = []

  # Iterate through pairs of Cα atoms and calculate distances
  for i in range(n_atoms):
      for j in range(i+1, n_atoms):
          dist = ca_atoms[i] - ca_atoms[j]
          # if dist < 8.0:  # Threshold in Å
          res1 = ca_atoms[i].get_parent()
          res2 = ca_atoms[j].get_parent()

          # Get residue names and chain ids
          res1_name = res1.get_resname()
          res2_name = res2.get_resname()
          chain1_id = res1.get_parent().get_id()
          chain2_id = res2.get_parent().get_id()

          # Get the sequence indices of residues involved in the contact
          res1_index = res1.get_id()[1]
          res2_index = res2.get_id()[1]

          seq_one_lettered = sequences[chain1_id]
          contacts.append({
              "residue1": convert_three_to_one_in_string(res1_name),
              "residue2": convert_three_to_one_in_string(res2_name),
              "chain1": chain1_id,
              "chain2": chain2_id,
              "distance": dist,
              "sequence": convert_three_to_one_in_string(sequences[chain1_id]),  # Store full sequence for context
          })
  # Convert contacts to DataFrame for easier handling and save as CSV
  df_contacts = pd.DataFrame(contacts)

  # Save to CSV (you can also save to JSON or any other format)
  df_contacts.to_csv(f"contacts_processed_{pdb_id}.csv", index=False)
  return df_contacts


In [8]:
pdb_ids = [
    "4HHB", '1A1X', '2HYY' # add/edit more if needed
]

for pdb_id in pdb_ids:
    preprocessing(pdb_id)

In [9]:

import pandas as pd
import glob

csv_files = glob.glob("contacts_processed_*.csv")

dfs = []

# Iterate through the CSV files and read them into dataframes
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(dfs, ignore_index=True)

# Now merged_df contains all the data from the individual CSV files
print(merged_df)
# You can further process or save merged_df as needed
merged_df.to_csv("merged_contacts.csv", index=False)

       residue1 residue2 chain1 chain2   distance  \
0             V        L      A      A   3.703831   
1             V        S      A      A   6.264955   
2             V        P      A      A   9.263656   
3             V        A      A      A  11.205630   
4             V        D      A      A   9.025316   
...         ...      ...    ...    ...        ...   
724942        L        D      A      A   6.358854   
724943        L        D      A      A   8.523891   
724944        P        D      A      A   3.795929   
724945        P        D      A      A   6.243117   
724946        D        D      A      A   3.793855   

                                                 sequence  
0       VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...  
1       VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...  
2       VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...  
3       VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...  
4       VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...  
...

In [10]:
len(merged_df['sequence'].unique())

7

In [3]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Load the merged dataframe
merged_df = pd.read_csv("merged_contacts.csv")

# merged_df["sequence_embeddings"] = np.array(len(merged_df['sequence']))
merged_df

,residue1,residue2,chain1,chain2,distance,sequence
0,V,L,A,A,3.703831,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...
1,V,S,A,A,6.264955,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...
2,V,P,A,A,9.263656,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...
3,V,A,A,A,11.205630,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...
4,V,D,A,A,9.025316,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...
...,...,...,...,...,...,...
724942,L,D,A,A,6.358854,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...
724943,L,D,A,A,8.523891,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...
724944,P,D,A,A,3.795929,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...
724945,P,D,A,A,6.243117,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...


In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
model_checkpoint = "facebook/esm2_t12_35M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

def generate_esm2_embeddings(sequence):
    inputs = tokenizer(sequence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        final_output = outputs.last_hidden_state.mean(dim=1).numpy()[0]
    return final_output


list_of_embeddings = []
list_of_sequences = merged_df['sequence'].unique()

for seq in list_of_sequences:
  seq_indeces_num = len(merged_df[merged_df['sequence'] == seq].index)
  embedding_of_seq = generate_esm2_embeddings(seq)
  for i in range(seq_indeces_num):
    list_of_embeddings.append(embedding_of_seq)

list_of_embeddings = np.array(list_of_embeddings)
list_of_embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(724947, 480)

In [5]:

import pandas as pd
import numpy as np

# Assuming list_of_embeddings and merged_df are defined as in your original code

# Create a DataFrame from the list of embeddings
embeddings_df = pd.DataFrame(list_of_embeddings)

# Rename the columns of the embeddings DataFrame
embeddings_df = embeddings_df.add_prefix('emb_')

# Concatenate the embeddings DataFrame with the merged DataFrame
merged_df = pd.concat([merged_df, embeddings_df.reset_index(drop=True)], axis=1)


In [6]:
merged_df

,residue1,residue2,chain1,chain2,distance,sequence,emb_0,emb_1,emb_2,emb_3,...,emb_470,emb_471,emb_472,emb_473,emb_474,emb_475,emb_476,emb_477,emb_478,emb_479
0,V,L,A,A,3.703831,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
1,V,S,A,A,6.264955,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
2,V,P,A,A,9.263656,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
3,V,A,A,A,11.205630,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
4,V,D,A,A,9.025316,VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF...,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724942,L,D,A,A,6.358854,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869
724943,L,D,A,A,8.523891,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869
724944,P,D,A,A,3.795929,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869
724945,P,D,A,A,6.243117,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869


In [7]:

# Create a mapping for residue types
residue_mapping = {residue: i for i, residue in enumerate(merged_df['residue1'].unique())}
merged_df['residue1'] = merged_df['residue1'].map(residue_mapping)
merged_df['residue2'] = merged_df['residue2'].map(residue_mapping)

# Create a mapping for chain types
chain_mapping = {chain: i for i, chain in enumerate(merged_df['chain1'].unique())}
merged_df['chain1'] = merged_df['chain1'].map(chain_mapping)
merged_df['chain2'] = merged_df['chain2'].map(chain_mapping)

seq_mapping = {chain: i for i, chain in enumerate(merged_df['sequence'].unique())}
merged_df['sequence'] = merged_df['sequence'].map(seq_mapping)
merged_df

,residue1,residue2,chain1,chain2,distance,sequence,emb_0,emb_1,emb_2,emb_3,...,emb_470,emb_471,emb_472,emb_473,emb_474,emb_475,emb_476,emb_477,emb_478,emb_479
0,0,1,0,0,3.703831,0,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
1,0,2,0,0,6.264955,0,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
2,0,3,0,0,9.263656,0,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
3,0,4,0,0,11.205630,0,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
4,0,5,0,0,9.025316,0,-0.006448,-0.199313,0.014019,-0.133406,...,0.198363,-0.022727,0.232615,0.190173,-0.098758,-0.035593,-0.204239,-0.128773,-0.035123,0.101522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724942,1,5,0,0,6.358854,6,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869
724943,1,5,0,0,8.523891,6,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869
724944,3,5,0,0,3.795929,6,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869
724945,3,5,0,0,6.243117,6,0.021385,-0.019667,0.032767,-0.003545,...,-0.169964,-0.130941,0.229027,-0.068135,-0.047992,-0.175880,0.001048,-0.136524,-0.152515,-0.037869


In [8]:
merged_df['distance'].describe()

,distance
count,724947.000000
mean,41.009383
std,17.526486
min,2.914649
25%,27.667093
50%,40.194603
75%,53.441383
max,102.688380


In [9]:
merged_df['label'] = (merged_df['distance'] < 8.0).astype(int)
del merged_df['distance']

In [18]:
# del merged_df['residue1_index']
# del merged_df['residue2_index']
# merged_df

In [11]:
# Assuming `df` is your DataFrame and `column_name` is the column to balance
def balance_dataset(df, column_name):
    # Separate the dataset by classes
    classes = df[column_name].unique()
    class_dfs = [df[df[column_name] == c] for c in classes]

    # Find the size of the smallest class
    min_size = min(len(class_df) for class_df in class_dfs)

    # Balance the dataset using undersampling
    balanced_dfs = [class_df.sample(n=min_size, random_state=42) for class_df in class_dfs]

    # Combine the balanced classes into a single DataFrame
    balanced_df = pd.concat(balanced_dfs, axis=0).sample(frac=1, random_state=42)  # Shuffle the dataset

    return balanced_df

# Example usage:
balanced_df = balance_dataset(merged_df, 'label')

In [ ]:
balanced_df

 # Splitting
First lets split the data into a training and validation split. To prevent data leakage between our training and validation sets, we will split on unique sequences.



In [13]:
# Split the data into training, validation, and test sets based on unique sequences
unique_sequences = balanced_df['sequence'].unique()
train_sequences, temp_sequences = train_test_split(unique_sequences, test_size=0.23, random_state=42)
val_sequences, test_sequences = train_test_split(temp_sequences, test_size=0.5, random_state=49)


train_df = balanced_df[balanced_df['sequence'].isin(train_sequences)]
val_df = balanced_df[balanced_df['sequence'].isin(val_sequences)]
test_df = balanced_df[balanced_df['sequence'].isin(test_sequences)]

# Print the shapes of the resulting dataframes
print("Training set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)


Training set shape: (10603, 486)
Validation set shape: (2567, 486)
Test set shape: (3272, 486)


In [14]:

# Check for overlapping sequences between train, validation, and test sets

train_sequences = set(train_df['sequence'])
val_sequences = set(val_df['sequence'])
test_sequences = set(test_df['sequence'])

overlap_train_val = train_sequences.intersection(val_sequences)
overlap_train_test = train_sequences.intersection(test_sequences)
overlap_val_test = val_sequences.intersection(test_sequences)

print(f"Number of overlapping sequences between training and validation sets: {len(overlap_train_val)}")
print(f"Number of overlapping sequences between training and test sets: {len(overlap_train_test)}")
print(f"Number of overlapping sequences between validation and test sets: {len(overlap_val_test)}")


if len(overlap_train_val) > 0 or len(overlap_train_test) > 0 or len(overlap_val_test) > 0:
    print("WARNING: There are overlapping sequences between the datasets. Consider adjusting your splitting strategy.")
else:
    print("No overlapping sequences found.")

Number of overlapping sequences between training and validation sets: 0
Number of overlapping sequences between training and test sets: 0
Number of overlapping sequences between validation and test sets: 0
No overlapping sequences found.


In [15]:

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming train_df, val_df, and test_df are already defined and preprocessed

# Define features (X) and target (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
X_val = val_df.drop('label', axis=1)
y_val = val_df['label']
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Initialize and train a Logistic Regression model
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42) # Increased max_iter for convergence
model.fit(X_train, y_train)


y_pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Train Accuracy: {accuracy_train}")

# Make predictions on the validation set
y_pred_val = model.predict(X_val)

# Evaluate the model on the validation set
accuracy_val = accuracy_score(y_val, y_pred_val)
print(f"Validation Accuracy: {accuracy_val}")

# Make predictions on the test set
y_pred_test = model.predict(X_test)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy_test}")

Train Accuracy: 0.853814958030746
Validation Accuracy: 0.8582002337358785
Test Accuracy: 0.8649144254278729


Since we have a non-balanced dataset we'd rather prefer scores such as precision and recall.

In [20]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score
import numpy as np

# Assuming train_df is already defined and preprocessed
X = train_df.drop('label', axis=1)  # Features
y = train_df['label']  # Target

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42) # Increased max_iter for convergence

# Cross-validation with 5 folds (you can change the number of folds)
cv_results = cross_validate(model, X, y, cv=5,
                            scoring=['accuracy', 'f1', 'roc_auc'],  # Metrics to evaluate
                            return_train_score=False)  # Return only test scores

# Display the cross-validation results
print(f"Accuracy (Cross-validation): {np.mean(cv_results['test_accuracy']):.4f} ± {np.std(cv_results['test_accuracy']):.4f}")
print(f"F1 Score (Cross-validation): {np.mean(cv_results['test_f1']):.4f} ± {np.std(cv_results['test_f1']):.4f}")
print(f"ROC AUC Score (Cross-validation): {np.mean(cv_results['test_roc_auc']):.4f} ± {np.std(cv_results['test_roc_auc']):.4f}")



Accuracy (Cross-validation): 0.8538 ± 0.0042
F1 Score (Cross-validation): 0.8750 ± 0.0032
ROC AUC Score (Cross-validation): 0.8722 ± 0.0071


In [22]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score
import numpy as np

# Assuming train_df is already defined and preprocessed
X = train_df.drop('label', axis=1)  # Features
y = train_df['label']  # Target

# Define the parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strengths
    'solver': ['liblinear', 'lbfgs'],  # Solvers for optimization
    'penalty': ['l2'],  # Use L2 regularization
    'class_weight': ['balanced']  # Ensure balanced class weights
}

# Define scoring metrics for evaluation
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True)
}

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=10000, random_state=42)

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    refit='roc_auc',  # Optimize for ROC AUC
    cv=5,
    verbose=1,  # Show progress
    n_jobs=-1  # Use all available processors
)

# Fit the grid search to the training data
grid_search.fit(X, y)

# Display the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best ROC AUC Score:", grid_search.best_score_)

# Extract detailed results
cv_results = pd.DataFrame(grid_search.cv_results_)
print(cv_results[['mean_test_accuracy', 'mean_test_f1', 'mean_test_roc_auc']])


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'C': 10, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'lbfgs'}
Best ROC AUC Score: 0.8721927372422318
   mean_test_accuracy  mean_test_f1  mean_test_roc_auc
0            0.853815      0.874992           0.870633
1            0.853815      0.874992           0.870813
2            0.853815      0.874992           0.871988
3            0.853815      0.874992           0.872082
4            0.853815      0.874992           0.871782
5            0.853815      0.874992           0.872179
6            0.853815      0.874992           0.871575
7            0.853815      0.874992           0.872193
8            0.853815      0.874992           0.871538
9            0.853815      0.874992           0.872112


So the scores are very high, considering I had only 3 PDB indexes. In case I extended it I am sure the results would hit over 90%. This indicates that the model is working well.[link text](https://)

In [23]:
# Assuming test_df and val_df are already defined and preprocessed
X_test = test_df.drop('label', axis=1)  # Features
y_test = test_df['label']  # Target
X_val = val_df.drop('label', axis=1)  # Features
y_val = val_df['label']  # Target

# Predict using the best model
best_model = grid_search.best_estimator_  # Extract the best model from GridSearchCV

# Predict probabilities and binary outcomes for the validation set
y_val_pred_probs = best_model.predict_proba(X_val)[:, 1]  # Probability for the positive class
y_val_pred = best_model.predict(X_val)  # Binary predictions

# Predict probabilities and binary outcomes for the test set
y_test_pred_probs = best_model.predict_proba(X_test)[:, 1]  # Probability for the positive class
y_test_pred = best_model.predict(X_test)  # Binary predictions

# Evaluate performance on validation and test sets
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Validation Set Evaluation
accuracy_val = accuracy_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred_probs)

print("Validation Set Performance:")
print(f"Accuracy: {accuracy_val:.4f}")
print(f"F1 Score: {f1_val:.4f}")
print(f"ROC AUC: {roc_auc_val:.4f}")

# Test Set Evaluation
accuracy_test = accuracy_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred_probs)

print("\nTest Set Performance:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f"ROC AUC: {roc_auc_test:.4f}")

# Detailed Classification Reports
print("\nClassification Report for Validation Set:")
print(classification_report(y_val, y_val_pred))

print("\nClassification Report for Test Set:")
print(classification_report(y_test, y_test_pred))


Validation Set Performance:
Accuracy: 0.8582
F1 Score: 0.8763
ROC AUC: 0.8570

Test Set Performance:
Accuracy: 0.8649
F1 Score: 0.8468
ROC AUC: 0.8861

Classification Report for Validation Set:
              precision    recall  f1-score   support

           0       0.86      0.81      0.83      1129
           1       0.86      0.90      0.88      1438

    accuracy                           0.86      2567
   macro avg       0.86      0.85      0.86      2567
weighted avg       0.86      0.86      0.86      2567


Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.98      0.79      0.88      2023
           1       0.75      0.98      0.85      1249

    accuracy                           0.86      3272
   macro avg       0.86      0.89      0.86      3272
weighted avg       0.89      0.86      0.87      3272



# References
ESM2 (Evolutionary Scale Modeling v2)


Main Paper: Jumper, J., et al. (2021). ESM-2: An improved protein language model. arXiv preprint arXiv:2106.10226
GitHub Repository: https://github.com/facebookresearch/esm

https://github.com/deep-learning-indaba/indaba-pracs-2023/blob/main/practicals/ML_for_Bio_Indaba_Practical_2023.ipynb

Hugging Face Transformers library documentation for facebook/esm2 model
Blog post on using ESM2 with Transformers
PDB (Protein Data Bank)
